In [1]:
import numpy as np

from metod_alg import metod_analysis as mt_ays
from metod_alg import objective_functions as mt_obj

import tqdm

In [111]:
func_name = 'hart'
d = 6
relax_sd = 1
num = 1
met = 'Brent'
option = 'minimize_scalar'
initial_guess = 0.005
projection = False
grad_norms = np.genfromtxt('%s_store_all_grad_norms_nsm_d=%s_%s_relax_c=%s_num=%s_%s.csv' %
                       (func_name, d, projection, relax_sd,
                        num, met),
                        delimiter=',')
no_its = np.genfromtxt('%s_store_all_its_nsm_d=%s_%s_relax_c=%s_num=%s_%s.csv' %
                       (func_name, d, projection, relax_sd,
                        num, met),
                        delimiter=',')
minimizer_store = np.genfromtxt('%s_all_store_minimizer_d=%s_%s_relax_c=%s_num=%s_%s.csv'%
                                (func_name, d, projection, relax_sd,
                                num, met),
                                delimiter=',')

In [112]:
beta = 0.01
total_pairs_nsm = np.genfromtxt('%s_beta=%s_tot_nsm_d=%s_%s_relax_c=%s_num=%s_%s.csv' %
                                (func_name, beta, d, projection, relax_sd,
                                num, met),
                                delimiter=',')

total_pairs_sm = np.genfromtxt('%s_beta=%s_tot_sm_d=%s_%s_relax_c=%s_num=%s_%s.csv' %
                                (func_name, beta, d, projection, relax_sd,
                                num, met),
                                delimiter=',')

In [113]:
total_pairs_sm

array([[286378., 286378.],
       [286378., 286378.]])

In [114]:
total_pairs_nsm

array([[208622., 208622.],
       [208622., 208622.]])

In [115]:
total_pairs_sm + total_pairs_nsm

array([[495000., 495000.],
       [495000., 495000.]])

In [116]:
# d = 6
# num_points = 100
# f = mt_obj.hartmann6_func
# g = mt_obj.hartmann6_grad
# check_func = mt_obj.calc_minimizer_hartmann6
# a, c, p = mt_obj.hartmann6_func_params()
# func_args = d, a, c, p
# func_args_check_func = ()
# bounds_1 = 0
# bounds_2 = 1
# number_its_compare = 3
# num = 1
# usage = 'metod_algorithm'
# relax_sd_it = 1
# tolerance = 0.01

In [117]:
np.round(np.mean(no_its), 2)

148.89

In [118]:
np.round(np.mean(grad_norms), 5)

1.8201

In [119]:
no_its

array([[601.,  15., 574., ...,  17., 545.,  23.],
       [ 14.,  15., 455., ...,  27.,  20., 376.],
       [ 21.,  25.,  18., ...,  31., 527.,  17.],
       ...,
       [513.,  18.,  57., ...,  18., 176.,  14.],
       [ 18.,  18., 495., ...,  23., 561., 592.],
       [486., 586., 449., ..., 602.,  17.,  18.]])

In [120]:
minimizer_store

array([[1., 0., 1., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 1.],
       [1., 1., 1., ..., 1., 0., 0.]])

In [121]:
store_dists_0 = []
store_grads_0 = []
store_dists_1 = []
store_grads_1 = []
for j in range(100):
    for i in range(100):
        if minimizer_store[j, i] == 0:
            store_dists_0.append(no_its[j, i])
            store_grads_0.append(grad_norms[j, i])
        else:
            store_dists_1.append(no_its[j, i])
            store_grads_1.append(grad_norms[j, i])

In [122]:
np.mean(store_dists_0)

20.294033275960988

In [123]:
np.mean(store_dists_1)

444.9910832232497

In [124]:
np.mean(store_grads_0)

1.6743570489283275

In [125]:
np.mean(store_grads_1)

2.1556691735677616

In [138]:
from numpy import linalg as LA
from metod_alg import objective_functions as mt_obj

In [145]:
np.random.seed(90)
d = 100
lambda_1 = 1
lambda_2 = 10
beta = 0.2
for i in range(1000):
    diag_vals = np.zeros(d)
    diag_vals[:2] = np.array([lambda_1, lambda_2])
    diag_vals[2:] = np.random.uniform(lambda_1 + 0.1,
                                      lambda_2 - 0.1, (d - 2))
    A = np.diag(diag_vals)
    rotation= mt_obj.calculate_rotation_matrix(d, 3)
    matrix_test = (np.transpose(rotation) @ A @
                   rotation)
    test = np.identity(d) - (np.identity(d) - beta * matrix_test)**2
    eigenvalues, eigenvectors = LA.eig(test)
    assert(np.all(eigenvalues >= 0))

In [47]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import tqdm

from metod_alg import objective_functions as mt_obj


def approx_beta(test, d):
    return (1 / np.mean(test)) *(((d - 1) * (d - 1)) / (2.25 * d * np.sqrt(d)))


def numerical_quad_exp(P, d_list, lambda_2, seed):
    f = mt_obj.several_quad_function
    g = mt_obj.several_quad_gradient
    lambda_1 = 1
    num = 100
    num_func = 100
    store_quantity = np.zeros((len(d_list), num_func))
    index = 0
    for d in d_list:
        for i in range(num_func):
            store_x0, store_A = mt_obj.function_parameters_several_quad(P, d, lambda_1, lambda_2)
            args = P, store_x0, store_A
            store_norm_grad = np.zeros((num ))
            for j in range(num):
                grad = g(np.random.uniform(0,1,(d,)), *args)
                store_norm_grad[j] = np.linalg.norm(grad)
            store_quantity[index, i] = approx_beta(store_norm_grad, d)
        index += 1
    return store_quantity



def func_parameters(func_name):
    """
    Generates function parameters for a particular function.

    Parameters
    ----------
    func_name : string
                Name of function.

    Returns
    --------
    d : integer
        Size of dimension.
    g : gradient of objective function.

       `g(x, *func_args) -> 1-D array with shape (d, )`

        where `x` is a 1-D array with shape(d, ) and func_args is a
        tuple of arguments needed to compute the gradient.
    func_args : tuple
                Arguments passed to f and g.
    bound_1 : integer
               Lower bound used to generate starting points.
    bound_2 : integer
               Upper bound used to generate starting points.

    """
    if func_name == 'styb':
        d = 5
        g = mt_obj.styblinski_tang_gradient
        func_args = ()
        bounds = (-5, 5)

    elif func_name == 'qing':
        d = 5
        g = mt_obj.qing_gradient
        func_args = (d,)
        bounds = (-3, 3)

    elif func_name == 'zak':
        d = 10
        g = mt_obj.zakharov_grad
        func_args = (d,)
        bounds = (-5, 10)
    return d, g, func_args, bounds



def compute_approx_unknown_lambda_max(g, args, bounds, d, num_func, num):
    store_quantity = np.zeros((num_func))
    index = 0
    for i in range(num_func):
        store_norm_grad = np.zeros((num))
        for j in range(num):
            grad = g(np.random.uniform(*bounds, (d,)), *args)
            store_norm_grad[j] = np.linalg.norm(grad)
        store_quantity[i] = approx_beta(store_norm_grad, d)
    return store_quantity


def boxplots_unknown_lambda_max():
    """
    Boxplots for approximation of beta.

    Parameters
    ----------
    store_all_norm_of_grad : 3-D array with shape (len(func_name_list),
                             num_functions, num_points).
                            The norm of the gradient at each starting point
                            for each function.
    num_functions : integer
                    Number of functions.
    """
    func_name_list = ['styb', 'qing', 'zak']
    num_func = 100
    num = 100
    store_quantities = np.zeros((len(func_name_list), num_func))
    index = 0
    for func_name in func_name_list:
        d, g, func_args, bounds = func_parameters(func_name)
        store_quantities[index] = (compute_approx_unknown_lambda_max(
                                   g, func_args, bounds, d, num_func, num))
        index += 1
    return store_quantities


seed = 90
type_func = 'unknown'
if type_func == 'known':
    lambda_list = [4, 10, 20]
    if P == 5:
        d_list = [5, 20, 100]
    elif P == 20:
        d_list = [20, 50, 100]
    elif P == 50:
        d_list = [50, 100, 200]
    store_all_quantity = np.zeros((len(lambda_list), len(d_list), 100))
    index = 0
    for lambda_2 in tqdm.tqdm(lambda_list):
        store_all_quantity[index] = numerical_quad_exp(P, d_list, lambda_2,
                                                    seed)
        index += 1
    ticks = d_list
    labels = [r'$\lambda_{max} = 4$',
            r'$\lambda_{max} = 10$',
            r'$\lambda_{max} = 20$']
    create_boxplots(store_all_quantity[0],
                    store_all_quantity[1],
                    store_all_quantity[2],
                    labels, ticks, P, seed)
elif type_func == 'unknown':
    store_quantities = boxplots_unknown_lambda_max()

In [49]:
np.median(store_quantities[0])

0.00582001198570449

In [50]:
np.median(store_quantities[1])

0.012724432696707144

In [51]:
np.median(store_quantities[2])

4.2510398279391696e-08